<a href="https://colab.research.google.com/github/Zhengzheren77/CIFAR10-Classification/blob/main/CIFAR_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CIFAR-10 Classification**

# 1. Data Processing

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import array_to_img, load_img
from tensorflow.keras.utils import img_to_array
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
N_CLASSES = 10

In [ ]:
#load dataset
(input_X_train, output_y_train), (input_X_test, output_y_test) = cifar10.load_data()
print('input_X_train shape:', input_X_train.shape)
print(input_X_train.shape[0], 'train samples')
print(input_X_test.shape[0], 'test samples')
 
# convert to categorical
output_Y_train = utils.to_categorical(output_y_train, N_CLASSES)
output_Y_test = utils.to_categorical(output_y_test, N_CLASSES) 

# float and normalization
input_X_train = input_X_train.astype('float32')
input_X_test = input_X_test.astype('float32')
input_X_train /= 255
input_X_test /= 255

input_X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
from sklearn.utils import shuffle

In [ ]:
x_train,y_train = shuffle(input_X_train, output_Y_train)
x_test,y_test = shuffle(input_X_test, output_Y_test)

In [ ]:
# Using 80-20 rule to get validation set
split = len(x_test)*80//100

x_train,x_valid = x_train[split:],x_train[:split]
y_train,y_valid = y_train[split:],y_train[:split]

In [ ]:
print('Train X : ',x_train.shape)
print('Train y :',y_train.shape)

print('1st training image shape ',x_train[0].shape)

print('Validation X : ',x_valid.shape)
print('Validation y :',y_valid.shape)
print('Test X : ',x_test.shape)
print('Test y : ',y_test.shape)

Train X :  (42000, 32, 32, 3)
Train y : (42000, 10)
1st training image shape  (32, 32, 3)
Validation X :  (8000, 32, 32, 3)
Validation y : (8000, 10)
Test X :  (10000, 32, 32, 3)
Test y :  (10000, 10)


# 2. Convolutional Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers.legacy import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras import backend as K
import time

In [ ]:
optimizer1 = Adam()
optimizer2 = SGD()
optimizer3 = RMSprop()

In [ ]:
batch_size = [16, 32, 64, 128]

In [ ]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)

## 2.1 One VGG block

In [ ]:
model1 = tf.keras.Sequential([
    Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(32, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.5),

    Dense(10, activation = 'softmax')
])


model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 128)               1048704   
                                                        

In [ ]:
ops = [optimizer1, optimizer2, optimizer3]
op_name = ['Adam', 'SGD', 'RMSProp']

In [ ]:
batch_size

[16, 32, 64, 128]

In [ ]:
model1_summary = pd.DataFrame()
path = '/content/drive/MyDrive/CNN_project/'
x = 1
for i in [64,128]: 
  for op in range(3):
    for lr in [0.05, 0.025, 0.005, 0.001]:
      print('Model:' + str(x))
      model1 = tf.keras.Sequential([
      Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
      Conv2D(32, (3, 3), padding = "same", activation='relu'),
      MaxPooling2D(pool_size = (2, 2)),
      Dropout(0.25),

      Flatten(),
      Dense(128, activation = 'relu'),
      Dropout(0.5),

      Dense(10, activation = 'softmax')
      ])
      model1.compile(optimizer = ops[op], loss='categorical_crossentropy', metrics=['accuracy'])
      K.set_value(model1.optimizer.learning_rate, lr)

      start_time = time.time()
      history_model = model1.fit(x_train, y_train, batch_size = i, epochs = 50, validation_data = (x_valid, y_valid), verbose = 1, callbacks = [es])
      end_time = time.time()


      history_df = pd.DataFrame(history_model.history)
      history_df['time'] = end_time-start_time
      history_df['optimization'] = op_name[op]
      history_df['batch_size'] = i
      history_df['learning_rate'] = lr
      x +=1

      model1_summary = pd.concat([model1_summary, history_df], axis =0)
    model1_summary.to_csv(Path(path+'bs_'+str(i)+'_'+op_name[op]+'.csv'))
    model1_summary = pd.DataFrame()

Model:1
Epoch 1/50
657/657 [==============================] - 15s 9ms/step - loss: 2.8719 - accuracy: 0.1008 - val_loss: 2.3084 - val_accuracy: 0.0925
Epoch 2/50
657/657 [==============================] - 4s 6ms/step - loss: 2.3070 - accuracy: 0.1003 - val_loss: 2.3055 - val_accuracy: 0.0989
Epoch 3/50
657/657 [==============================] - 4s 6ms/step - loss: 2.3062 - accuracy: 0.1035 - val_loss: 2.3097 - val_accuracy: 0.0966
Epoch 4/50
657/657 [==============================] - 5s 8ms/step - loss: 2.3075 - accuracy: 0.1005 - val_loss: 2.3053 - val_accuracy: 0.0994
Epoch 5/50
657/657 [==============================] - 4s 6ms/step - loss: 2.3069 - accuracy: 0.0979 - val_loss: 2.3043 - val_accuracy: 0.0966
Epoch 6/50
657/657 [==============================] - 4s 6ms/step - loss: 2.3075 - accuracy: 0.1015 - val_loss: 2.3110 - val_accuracy: 0.0984
Epoch 7/50
657/657 [==============================] - 7s 10ms/step - loss: 2.3068 - accuracy: 0.1045 - val_loss: 2.3037 - val_accuracy: 0.1

## 2.2 Two VGG Blocks

In [ ]:
model2 = tf.keras.Sequential([
    Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(32, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(64, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),   
    Dropout(0.25),  

    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.5),

    Dense(10, activation = 'softmax')
])


model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                        

In [ ]:
model2_summary = pd.DataFrame()
x = 1
for i in batch_size:
  for op in [optimizer1, optimizer2, optimizer3]:
    for lr in [0.05, 0.025, 0.005, 0.001]:
      print('Model: ' + str(x))
      model1 = tf.keras.Sequential([
      Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
      Conv2D(32, (3, 3), padding = "same", activation='relu'),
      MaxPooling2D(pool_size = (2, 2)),
      Dropout(0.25),

      Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
      Conv2D(64, (3, 3), padding = "same", activation='relu'),
      MaxPooling2D(pool_size = (2, 2)),
      Dropout(0.25),

      Flatten(),
      Dense(128, activation = 'relu'),
      Dropout(0.5),

      Dense(10, activation = 'softmax')
      ])
      model2.compile(optimizer = op, loss='categorical_crossentropy', metrics=['accuracy'])
      K.set_value(model2.optimizer.learning_rate, lr)

      start_time = time.time()
      history_model = model2.fit(x_train, y_train, batch_size = i, epochs = 50, validation_data = (x_valid, y_valid), verbose = 1, callbacks = [es])
      end_time = time.time()


      history_df = pd.DataFrame(history_model.history)
      history_df['time'] = end_time-start_time
      history_df['optimization'] = op
      history_df['batch_size'] = i
      history_df['learning_rate'] = lr

      model2_summary = pd.concat([model2_summary, history_df], axis =0)
      x +=1 

## 2.3 Three VGG Blocks

In [ ]:
model3 = tf.keras.Sequential([
    Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(32, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(64, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),   
    Dropout(0.25),  

    Conv2D(128, (3, 3), padding = "same", activation='relu', input_shape = (32, 32, 3)),
    Conv2D(128, (3, 3), padding = "same", activation='relu'),
    MaxPooling2D(pool_size = (2, 2)),   
    Dropout(0.25),     

    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.5),

    Dense(10, activation = 'softmax')
])


model3.summary()